In [1]:
from base.base_inference import VideoInference
from models import UNet
from models import DeepLabV3Plus
from subprocess import call
import torch
import ffmpeg

In [16]:
# CHECKPOINT = "./pretr/DeepLabV3Plus_ResNet18.pth"
CHECKPOINT = "pretr/model_best.pth"
BACKBONE   = "resnet18"
VIDEO_INP  = "./vd/5.mp4"
VIDEO_OUT  = "./vd/out.mp4"
VIDEO_TMP  = "./vd/.tmp.mp4"

In [19]:
!nvidia-smi

Wed Mar 25 18:36:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0  On |                  N/A |
| 26%   43C    P8    23W / 250W |    121MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   38C    P8    10W / 250W |    727MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
torch.cuda.set_device(1)
torch.cuda.current_device()

1

In [4]:
model = DeepLabV3Plus(backbone=BACKBONE, num_classes=2)
trained_dict = torch.load(CHECKPOINT, map_location="cpu")['state_dict']
model.load_state_dict(trained_dict, strict=False)
model.cuda()
model.eval()
print('ok')

[ResNet] Initialize weights...
ok


In [17]:
call(["rm", VIDEO_TMP])
inference = VideoInference(
    model=model,
    video_path=VIDEO_INP,
    video_out_path=VIDEO_TMP,
    input_size=480,
    background_path = "./backgrounds/white.jpg",
    use_cuda=True,
    draw_mode='matting'
    # frame_range=(0, 1000)
)
inference.run()

100% (2271 of 2271) |####################| Elapsed Time: 0:02:45 Time:  0:02:45


In [18]:
in1 = ffmpeg.input(VIDEO_INP)
in2 = ffmpeg.input(VIDEO_TMP)
out = ffmpeg.output(in1.audio, in2.video, VIDEO_OUT)
out.run(overwrite_output=True)

(None, None)